In [ ]:
from pyspark.sql.session import SparkSession
from pyspark.sql.types import BooleanType
from pyspark.sql.functions import expr
from pyspark.sql import functions as F
from pyspark.ml.stat import Summarizer
from pyspark.ml.feature import VectorAssembler, StringIndexer, StandardScaler
from helpers.helper_functions import translate_to_file_string

In [ ]:
inputFile = translate_to_file_string("../data/Flight_Delay_Jan_2020_ontime.csv")

In [ ]:
spark = (SparkSession
       .builder
       .appName("FlightDelay")
       .getOrCreate())

In [ ]:
pysparkDF = spark.read.option("header", "true") \
        .option("inferSchema", "true") \
        .option("delimiter", ",") \
        .csv(inputFile)

pysparkDF.printSchema()

### Remove faulty features

In [ ]:
pysparkDF = pysparkDF.drop('_c21')
pysparkDF.printSchema()

### Remove records containing NULL values

In [ ]:
pysparkDF_nonull = pysparkDF.dropna()
f"Removed {pysparkDF.count()-pysparkDF_nonull.count()} records containing NULL values"

### Build String indexer for TAIL_NUM

In [ ]:
tailNum_Indexer = StringIndexer().setInputCol("TAIL_NUM").setOutputCol("TAIL_NUM_ID").fit(pysparkDF_nonull)
pysparkDF_indexed = tailNum_Indexer.transform(pysparkDF_nonull)

### Define label columns

In [ ]:
labelCols = ["DEP_DEL15","ARR_DEL15"]
labelCols

### Remove redundant features and labels for unconditional prediction
-> Unconditional is referring to predicting each of the labels without having information on the current status of the flight (Use-Case: Checking the day before)

In [ ]:
# Remove strings from id/string pairs (redundant)
# Remark: since in this dataset both string and id exist already, no further preprocessing via string_indexer is necessary. Otherwise, strings would have first been converted to ids via string_indexer.
featureCols_unconditional = pysparkDF_indexed.columns.copy()
featureCols_unconditional.remove("TAIL_NUM") # -> TAIL_NUM_ID
featureCols_unconditional.remove("OP_UNIQUE_CARRIER") # -> OP_CARRIER_AIRLINE_ID
featureCols_unconditional.remove("OP_CARRIER") # -> OP_CARRIER_AIRLINE_ID
featureCols_unconditional.remove("ORIGIN") # -> ORIGIN_AIRPORT_ID
featureCols_unconditional.remove("ORIGIN_AIRPORT_SEQ_ID") # -> ORIGIN_AIRPORT_ID
featureCols_unconditional.remove("DEST") # -> DEST_AIRPORT_SEQ_ID
featureCols_unconditional.remove("DEST_AIRPORT_SEQ_ID") # -> DEST_AIRPORT_SEQ_ID
featureCols_unconditional.remove("DEP_TIME_BLK") # -> preliminary elimination, check if model works better with binned values or not

for label in labelCols:
    featureCols_unconditional.remove(label)
                                     
featureCols_unconditional                         

### Remove redundant features and labels for conditional prediction
-> Conditional is referring to predicting each of the labels considering available real-time information on the current status of the flight (Use-Case: Checking while at the airport, pre-flight)

One would expect that prediction performance is increased when the model is aware of the current flight status (=DEP_DEL15)

Example: If the model is aware that the flight has departure delay, it might be able to better predict whether it will also be delayed at arrival

In [ ]:
# Remove strings from id/string pairs (redundant)
# Remark: since in this dataset both string and id exist already, no further preprocessing via string_indexer is necessary. Otherwise, strings would have first been converted to ids via string_indexer.
featureCols_conditional = pysparkDF_indexed.columns.copy()
featureCols_conditional.remove("TAIL_NUM") # -> TAIL_NUM_ID
featureCols_conditional.remove("OP_UNIQUE_CARRIER") # -> OP_CARRIER_AIRLINE_ID
featureCols_conditional.remove("OP_CARRIER") # -> OP_CARRIER_AIRLINE_ID
featureCols_conditional.remove("ORIGIN") # -> ORIGIN_AIRPORT_ID
featureCols_conditional.remove("ORIGIN_AIRPORT_SEQ_ID") # -> ORIGIN_AIRPORT_ID
featureCols_conditional.remove("DEST") # -> DEST_AIRPORT_SEQ_ID
featureCols_conditional.remove("DEST_AIRPORT_SEQ_ID") # -> DEST_AIRPORT_SEQ_ID
featureCols_conditional.remove("DEP_TIME_BLK") # -> preliminary elimination, check if model works better with binned values or not

for label in [label for label in labelCols if label!="DEP_DEL15"]:
    featureCols_conditional.remove(label)
    
featureCols_conditional

### Build and apply feature column assembler for both featureCols

In [ ]:
assembler_unconditional =  VectorAssembler(outputCol="features", inputCols=list(featureCols_unconditional))
assembler_conditional =  VectorAssembler(outputCol="features", inputCols=list(featureCols_conditional))

featureSet_unconditional = assembler_unconditional.transform(pysparkDF_indexed)
featureSet_conditional = assembler_conditional.transform(pysparkDF_indexed)

# Define same base-scaler for both feature cols
scaler = StandardScaler(inputCol="features",
                        outputCol="scaledFeatures",
                        withStd=True, 
                        withMean=False)

# Compute summary statistics by fitting the StandardScaler
scalerModel_unconditional = scaler.fit(featureSet_unconditional)
scalerModel_conditional = scaler.fit(featureSet_conditional)

scaledFeatureSet_unconditional = scalerModel_unconditional.transform(featureSet_unconditional)
scaledFeatureSet_conditional = scalerModel_conditional.transform(featureSet_conditional)

## Modelling
### Split data into training and test set
Die Aufteilung der Daten erfolgt in 80% Trainingsdaten und 20% Testdaten.

In [ ]:
splits_unconditional = scaledFeatureSet_unconditional.randomSplit([0.8, 0.2], 12345)
training_unconditional = splits_unconditional[0]
test_unconditional = splits_unconditional[1]

splits_conditional= scaledFeatureSet_conditional.randomSplit([0.8, 0.2], 12345)
training_conditional = splits_conditional[0]
test_conditional = splits_conditional[1]

In [ ]:
training_unconditional.printSchema()

### Build and train the Logistic Regression Model

Um eine Logistische Regression durchführen zu können, muss zunächst das zugehörige Element `LogisticRegression` aus der Bibliothek `pyspark.ml.classification` importiert werden.

In [ ]:
from pyspark.sql.types import *
schema = StructType([
      StructField('model', StringType(), True),
      StructField('target label', StringType(), True),
      StructField('mode', StringType(), True),
      StructField('param_config', StringType(), True),
      StructField('accuracy', FloatType(), True),
      StructField('test error', FloatType(), True)
  ])

evalDF = spark.createDataFrame([], schema)
evalDF.show()

In [ ]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.mllib.evaluation import MulticlassMetrics


models = ["LogisticRegression"]

for model in models:
    for label in labelCols:
        for test, train, mode, features in zip([test_unconditional, test_conditional],[training_unconditional,training_conditional],["Unconditional","Conditional"],[featureCols_unconditional,featureCols_conditional]):
            # Skip invalid combinations
            if label=="DEP_DEL15" and mode=="Conditional":
                continue

            # Print Model Spec
            print("\n\n----------MODEL SPEC----------")
            print(f"Model Type: {model}")
            print(f"Target Label: {label}")
            print(f"Prediction Mode: {mode}")

            # Define Model
            if model=="LogisticRegression":
                # Define LogisticRegression Classifier acc. to current param
                model_instance = LogisticRegression(
                               featuresCol="scaledFeatures",
                               labelCol=label,
                               standardization=False)
                paramGrid = ParamGridBuilder().addGrid(model_instance.maxIter, [20])\
                                 .addGrid(model_instance.regParam, [0.7]) \
                                 .addGrid(model_instance.elasticNetParam, [1.0]) \
                                 .build()
                params = ["maxIter","regParam","elasticNetParam"]
            if model=="SVM":
                # Define SVM Classifier acc. to current param
                pass

            evaluator = BinaryClassificationEvaluator(labelCol=label)
            cv = CrossValidator(estimator=model_instance, evaluator=evaluator, \
                          estimatorParamMaps=paramGrid, numFolds=5, parallelism=2)
            cvModel = cv.fit(train)
            model_best = cvModel.bestModel
            param_print = '\n'.join([line for line in model_best.explainParams().split('\n') if line.split(":")[0] in params])
            print("Chosen parameters: \n" + param_print)

            print(str(model) + " Coefficients: " + str(dict(zip(features,["{:.4f}".format(a) for a in model_best.coefficients]))))
            print(str(model) + " Intercept: " + "{:.4f}".format(model_best.intercept))

            # Predict and evaluate
            predictions = cvModel.transform(test)
            predictionAndLabels = predictions.select(predictions.prediction, label)

            countcorrect = predictionAndLabels.filter(f"{label} == prediction").count()
            countincorrect = predictionAndLabels.filter(f"{label} != prediction").count()
            countall = predictionAndLabels.count()
            accuracy = countcorrect/countall
            print(f"count correct: {countcorrect}")
            print(f"count incorrect: {countincorrect}")
            print(f"count all: {countall}")
            print(f"accuracy: {accuracy}")
            print(f"Test Error {1-accuracy}")
            
            predictionAndLabels = predictions.select("prediction", label).rdd.map(lambda p: [p[0], float(p[1])]) # Map to RDD prediction|label
            metrics =  MulticlassMetrics(predictionAndLabels)
            confusion = metrics.confusionMatrix()            
            print("Confusion matrix: \n" , confusion)
            
            print("--------------------")

            newRow = spark.createDataFrame([(model,label,mode,param_print,accuracy,1-accuracy)], schema)
            evalDF = evalDF.union(newRow)

In [ ]:
evalDF.show(20,False)

In [ ]:
#spark.stop()